In [ ]:
import requests
from tqdm import tqdm

def llegir_fitxer(input_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            paraules = f.readlines()
        return paraules
    except FileNotFoundError:
        print(f"El fitxer {input_file} no existeix.")
        return []
    except Exception as e:
        print(f"Error llegint el fitxer {input_file}: {e}")
        return []

def comprovar_pagina_diec(paraula):
    paraula = paraula.strip()
    url = f'https://dlc.iec.cat/Results?DecEntradaText={paraula}&AllInfoMorf=False&OperEntrada=0&OperDef=0&OperEx=0&OperSubEntrada=0&OperAreaTematica=0&InfoMorfType=0&OperCatGram=False&AccentSen=False&CurrentPage=0&refineSearch=0&Actualitzacions=False'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return "No s'ha trobat cap entrada coincident amb els criteris de cerca" not in response.text
    except requests.RequestException as e:
        print(f"Error en la petició per a la paraula {paraula}: {e}")
        return False

def escriure_resultats(output_file, resultats):
    progress_bar = tqdm(total=len(resultats), desc="Escrivint resultats", unit=" elements")
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for resultat in resultats:
                f.writelines(resultat)
                progress_bar.update(1)

    except Exception as e:
        print(f"Error escrivint al fitxer {output_file}: {e}")
    
    progress_bar.close()


def executar_comprovacio(input_file, output_file):
    paraules = llegir_fitxer(input_file)
    if not paraules:
        print("No hi ha paraules per processar.")
        return

    resultats = []
    
    # Inicialitzem la barra de progrés 
    progress_bar = tqdm(total=len(paraules), desc="Comparant llistes", unit=" elements")

    for paraula in paraules:
        if comprovar_pagina_diec(paraula):
            resultats.append(paraula)
        progress_bar.update(1)

    progress_bar.close()
    
    escriure_resultats(output_file, resultats)

    print("Fet")

input_file = '../diccionaris/separat/col_1_sense_repetir.txt'
output_file = '../diccionaris/viccionari/paraules_sí_diec.txt'

executar_comprovacio(input_file, output_file)

In [ ]:
def eliminar_paraules_repetides(nom_fitxer_entrada, nom_fitxer_sortida):
    # Llegir totes les línies del fitxer d'entrada
    with open(nom_fitxer_entrada, 'r') as file:
        lines = file.readlines()
    
    # Eliminar duplicats mantenint l'ordre d'aparició
    paraules_uniques = list(dict.fromkeys([line.strip() for line in lines]))
    
    # Escriure les paraules úniques al fitxer de sortida
    with open(nom_fitxer_sortida, 'w') as file:
        for paraula in paraules_uniques:
            file.write(f"{paraula}\n")

# Exemple d'ús:
fitxer_entrada = '../diccionaris/separat/col_1.txt'
fitxer_sortida = '../diccionaris/separat/col_1_sense_repetir.txt'

eliminar_paraules_repetides(fitxer_entrada, fitxer_sortida)
